In [2]:
from __future__ import print_function
from visualizer.show3d_balls import showpoints
import argparse
import numpy as np
import glob
import easydict

import torch
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable

from data_utils.CustomDataLoader import PartNormalDataset
from models import pointnet_part_seg
import matplotlib.pyplot as plt

In [3]:
root = '/home/trojan/skia_projects/3d_facial_segmentation/part_segmentation/pointnet/Pointnet_Pointnet2_pytorch/data/shapenetcore_partanno_segmentation_benchmark_v0_normal/'
model_path = './log/part_seg/2021-08-11_14-37/checkpoints/best_model.pth'

num_part = 2
num_classes = 1
seg_classes = {'Head': [0, 1]}

In [4]:
args = easydict.EasyDict({'num_point': 2500,
                          'normal': False
                         })

In [5]:
d = PartNormalDataset(root=root, npoints=args.num_point, split='test', class_choice = ['Head'], normal_channel=args.normal)
print(len(d))
testDataLoader = torch.utils.data.DataLoader(d, batch_size=1, shuffle=False, num_workers=4)

1


In [6]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
    pc = pc / m
    return pc

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    new_y = torch.eye(num_classes)[y.cpu().data.numpy(),]
    if (y.is_cuda):
        return new_y.cuda()
    return new_y

In [7]:
# for pcd in glob.glob(testDir + '/*.txt'):
#     data = np.loadtxt(pcd).astype(np.float32)
#     point_set = data[:, 0:3]
#     point_set = pc_normalize(point_set)
#     seg = data[:, -1].astype(np.int32)
#     #print(len(point_set), len(seg))
    
#     cmap = plt.cm.get_cmap("hsv", 10)
#     cmap = np.array([cmap(i) for i in range(10)])[:, :3]
#     gt = cmap[seg - 1, :]
    
#     model = args.model
#     checkpoint = torch.load(model_path)
#     classifier = model.get_model(num_part, normal_channel=normal_channel)
#     classifier.load_state_dict(checkpoint['model_state_dict'])
#     classifier.eval()
    
#     for cat in seg_classes.keys():
#         for label in seg_classes[cat]:
#             seg_label_to_cat[label] = cat
                    
#     pred, _ = classifier(point_set, to_categorical(label, num_classes))
#     pred_choice = pred.data.max(2)[1]
#     print(pred_choice)

In [8]:
for points, label, target in testDataLoader:
    print(points.dtype)

torch.float32


In [22]:
checkpoint = torch.load(model_path)
classifier = pointnet_part_seg.get_model(num_part, normal_channel=args.normal)
classifier.load_state_dict(checkpoint['model_state_dict'])
classifier.eval()

seg_label_to_cat = {}

for cat in seg_classes.keys():
                for label in seg_classes[cat]:
                    seg_label_to_cat[label] = cat
                
for points, label, target in testDataLoader:
    cur_batch_size, NUM_POINT, _ = points.size()
    points_np = points.numpy()
    
    points = points.transpose(2, 1)
    
    cmap = plt.cm.get_cmap("hsv", 10)
    cmap = np.array([cmap(i) for i in range(10)])[:, :3]
    gt = cmap[target.numpy() - 1, :]

    seg_pred, _ = classifier(points, to_categorical(label, num_classes))
    cur_pred_val = seg_pred.cpu().data.numpy()
    cur_pred_val_logits = cur_pred_val
    cur_pred_val = np.zeros((cur_batch_size, NUM_POINT)).astype(np.int32)
    target = target.cpu().data.numpy()
    print(seg_pred)
    
    for i in range(cur_batch_size):
        cat = seg_label_to_cat[target[i, 0]]
        logits = cur_pred_val_logits[i, :, :]
        cur_pred_val[i, :] = np.argmax(logits[:, seg_classes[cat]], 1) + seg_classes[cat][0]
    
        print(cur_pred_val[i,:])
                
#     pred_choice = seg_pred.data.max(2)[1]
#     print(pred_choice)
    
#     pred_color = cmap[pred_choice.numpy()[0], :]
    
    
    #showpoints(points_np, gt, pred_color)
#     cur_pred_val = seg_pred.cpu().data.numpy()
#     cur_pred_val_logits = cur_pred_val
#     cur_pred_val = np.zeros((cur_batch_size, NUM_POINT)).astype(np.int32)
#     target = target.cpu().data.numpy()
    
#     for i in range(cur_batch_size):
#         cat = seg_label_to_cat[target[i, 0]]
#         logits = cur_pred_val_logits[i, :, :]
#         cur_pred_val[i, :] = np.argmax(logits[:, seg_classes[cat]], 1) + seg_classes[cat][0]
    
    
#     correct = np.sum(cur_pred_val == target)

tensor([[[-0.4318, -1.0479],
         [-0.0471, -3.0788],
         [-0.1109, -2.2542],
         ...,
         [-0.2078, -1.6732],
         [-0.0326, -3.4398],
         [-0.4144, -1.0810]]], grad_fn=<ViewBackward>)
[0 0 0 ... 0 0 0]


In [10]:
np.savetxt('/home/trojan/Desktop/pred_choice.txt', pred_choice)